# タイタニックの生存予測 提出ファイルの作成 <a name="top"></a>

このnotebookは[AutoAIでKaggle タイタニックの生存予測にチャレンジ](https://qiita.com/nishikyon/private/e55826c2bb6d38797628)に全体のガイドが書かれています。まずはこちらを参照してください。

このnotebookはIBMCloud上のWatson Studioのnotebookで動くように書かれています。

[Watson Studio上のnotebookからIBM Cloud Object Storage(ICOS)へのFileの読み書き - project-libを使う -](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40)の手順が前準備として必要です。

まずは1,2を実施お願いします。
-  [1. Projectを開く](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40)
-  [2. 前準備 Access tokenの作成](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#2-%E5%89%8D%E6%BA%96%E5%82%99-access-token%E3%81%AE%E4%BD%9C%E6%88%90)

その後再度このnotebookを開いて、Kaggleからダウンロードしたtest.csvを
-  [4: 読み込ませたいファイルをICOSにアップロード](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#4-%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BE%E3%81%9B%E3%81%9F%E3%81%84%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%92icos%E3%81%AB%E3%82%A2%E3%83%83%E3%83%97%E3%83%AD%E3%83%BC%E3%83%89)　の方法でアップロードします。

アップロードが完了したら
- [5: Project tokenのInsert](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#5-project-token%E3%81%AEinsert)を実施してください。

前準備は以上になります。

# Contents

1.  [Watson Machine Learning API clientの導入](#setup)
2.  [Watson Machine Learningの資格情報の取得](#credential)
3.  [Deployment名のセット](#set_deployment)　　　<br>
　（[notebookの実行]( #run_notebook) 3まで変更すると、あとはそのまま実行可能です。)
4.  [Deploymentのuidの取得](　#get_deployed_uid)
5.  [予測するデータファイルの読み込み](#load_test_csv)
6.  [NaNをNoneに変換,](#convert_nan)
7.  [DataFrameのデータをAPI用のパラメーターフォーマットに変換](#convert_dataframe)
8.  [予測結果の取得](#get_prediction)
9.  [提出用にDataFrameにしてフォーマット](#format_results
10.  [フォーマットしたDataFrameをICOS上にSave](#save_results)

# 1. Watson Machine Learning API clientの導入 <a name="setup"></a>
このNotebookでは[Watson Machine Learning Python client library, version 4](https://wml-api-pyclient-dev-v4.mybluemix.net/)を使用します。次のセルでpipで導入します。

参考:
- https://watson-ml-v4-api.mybluemix.net/<br/>
- https://wml-api-pyclient.mybluemix.net/

In [ ]:
# Install the Watson Machine Learning API client
!pip install watson-machine-learning-client-v4

# 2. Watson Machine Learningの資格情報の取得 <a name="credential"></a>

[ダッシュボード](https://cloud.ibm.com/)→[リソースの表示](https://cloud.ibm.com/resources)の`Services`の下からWatson Machine　Learningサービスの名前をクリックします。

詳細が作成されたら、画面左側の`サービス資格情報`リンクをクリックします。`新規資格情報`ボタンをクリックし、資格情報に名前を付けて、追加をクリックします。新しいクレデンシャルは、`資格情報の表示`ボタンをクリックしてアクセスできます。表示された資格情報をコピーして下のセルに貼り付けます。

In [ ]:
#自分のWatson Machine Learningの資格情報をペーストしてください。
wml_credentials = {
  "apikey": "<ここにコピペ>",
  "instance_id": "<ここにコピペ>",
  "url": "<ここにコピペ>"
}

# 3. Deployment名のセット <a name="set_deployment"></a>

下のセルを実行後、表示されたNAMEからTitanicのAutoAIで作成されたモデルのDeploymemnt名をそのさらに一つ下の`DEPLOYMENT_NAME`にセットし、実行する。

In [ ]:
# WatsonMachineLearningAPIClientの作成
from watson_machine_learning_client import WatsonMachineLearningAPIClient

client = WatsonMachineLearningAPIClient(wml_credentials)

# deployed model一覧の取得
client.deployments.list()

In [ ]:
#  以下に作成したDeployment名をセットする。
# Qiitaと同じに作成している場合はそのままでよい。
# 上のセルの実行結果のNAMEにリストされている名前と同じであることを確認
DEPLOYMENT_NAME = 'Titanic AutoAI Deployment'

# notebookの実行 <a name="run_notebook"></a>

これで変更が必要な部分の編集は全て完了しました。
説明を読まず以下のセルを全て実行してもよいし、説明を読みながら進めたい場合は、読みながらそのまま順に実行していってください。

# 4. Deploymentのuidの取得　<a name="get_deployed_uid"></a>
DEPLOYMENT_NAMEでセットしたDeploymentのuidを取得します。

エラーが発生する場合は上のセルで設定したDEPLOYMENT_NAMEが間違っている可能性がありますので、確認してください。

In [ ]:
# Extract the information for DEPLOYMENT_NAME
deployments_details = client.deployments.get_details()
deployed_uid = next(item for item in deployments_details['resources'] 
                    if item['entity']["name"] == DEPLOYMENT_NAME)['metadata']['guid']

# 5. 予測するデータファイルの読み込み　<a name="load_test_csv"></a>

予測するデータファイル`test.csv`をpandasのデータフレームに読み込みます。

[タイタニックの生存予測 提出ファイルの作成](#top)に書いてあるファイルを読むための前準備とアップロードを行なっていないとエラーになりますので、必ず前準備の実施を行なってください。

In [ ]:
#データ読み込み
csv_file_name = 'test.csv'
csv_file = project.get_file(csv_file_name)
csv_file.seek(0)

import pandas as pd
df_test = pd.read_csv(csv_file)
df_test.head()


# 6. NaNをNoneに変換 <a name="convert_nan"></a>

AutoAIにRESTでアクセスする際、pandasのNaNはNoneに変換しないとうまく動かないため、変換します。

In [ ]:
# NaNをNoneに変換
df_test = df_test.where((pd.notnull(df_test)), None)

df_test.head()

# 7. DataFrameのデータをAPI用のパラメーターフォーマットに変換　<a name="convert_dataframe"></a>

DataFrameに読み込んだデータをWatson Machine Learning API 用のパラメーターフォーマットに変換にします。

In [ ]:
feature_columns = df_test.columns.tolist()
values = df_test.values.tolist()
scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA: [{
    "fields": feature_columns, 
    "values": values
    }]}

#　確認のため最初の1000文字だけ表示
import json
print('{:.1000}'.format( json.dumps(scoring_payload)))

# 8. 予測結果の取得　<a name="get_prediction"></a>

APIでAutoAIで作成したモデルのDeploymentを呼び出し、予測結果を取得します。

In [ ]:
predictions = client.deployments.score(deployed_uid, scoring_payload)

In [ ]:
#　確認のため最初の1000文字だけ表示
print('{:.1000}'.format( json.dumps(predictions)))

#　確認のため結果の最初10行分だけ表示
display(predictions['predictions'][0]['values'][0:10])

# 9. 提出用にDataFrameにしてフォーマット　<a name="format_results"></a>

提出用のフォーマットサンプルはダンロードした`gender_submission.csv`と同じになります。
`PassengerId`と`Survived`のみが必要なので、`test.csv`(=DataFrame　`df_test`)の`PassengerId`と`predictions`に入っている結果の値をDataFrameにして`Survived`として結合します。

In [ ]:
df_submission = pd.DataFrame()
df_submission['PassengerId'] = df_test["PassengerId"]
df_survived =pd.DataFrame(predictions['predictions'][0]['values'], columns = ["Survived","probability"])[["Survived"]]
df_submission = df_submission.join(df_survived)

#　確認
display(df_submission)

# 10. フォーマットしたDataFrameをICOS上にSave　<a name="save_results"></a>

フォーマットしたDataFrameをICOS上にSaveします。これでファイルとしてダウンロードできるようになります。

In [ ]:
project.save_data("submission_titanic.csv",df_submission.to_csv(index=False),overwrite=True)

# これで提出できる準備ができました！

FileメニューからこのnotebookをSaveして、上部からProject名のLinkをクリックし、Projectページに戻ります。

`Assets`タブの`Data Sources`の下に`submission_titanic.csv`というファイルがあるはずなので、それをダウンロードしてください。

